In [1]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.4f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# pnu별 매출, 점포수, 유동인구, 거주인구 증감율
sql = 'select pnu, sale_12m, comp_12m, fltg_12m, lpop_12m, base_ym from m2.ibk_pnu_01'
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df = pd.DataFrame(rows)
df.columns = ['pnu','sale_12m','comp_12m','fltg_12m','lpop_12m','base_ym']

# pnu별 주용도명
sql = '''
select main_purps_cd_nm, sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji as pnu from m1.bld_rgst
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_bld_principal_use = pd.DataFrame(rows)
df_bld_principal_use.columns = ['principal_use','pnu']

In [2]:
df

,pnu,sale_12m,comp_12m,fltg_12m,lpop_12m,base_ym
0,1111010100100010000,0.9957,0.8889,1.2237,1.5722,202110
1,1111010100100030065,0.9569,1.5000,1.2237,1.5722,202110
2,1111010100100060003,1.1285,0.5000,3.2260,2.4503,202110
3,1111010100100150004,0.9467,1.0000,1.2237,1.5722,202110
4,1111010100100150005,0.9577,2.0000,1.2237,1.5722,202110
...,...,...,...,...,...,...
10736287,4137011300106020001,1.1168,0.5000,1.6687,1.2762,202209
10736288,4137011300106030002,0.9701,1.0000,1.6687,1.2762,202209
10736289,4137011300106050003,0.9656,1.0000,1.6687,1.2762,202209
10736290,4137011300106050006,0.9429,1.0000,1.6687,1.2762,202209


In [38]:
# 0/1 나누는 기준 : 중앙값 
df['sale'] = 0
idx = df[df['sale_12m'] > (round(df['sale_12m'].median(), 4))].index
df['sale'][idx] = '1'

df['comp'] = 0
idx = df[df['comp_12m'] > (round(df['comp_12m'].median(), 4))].index
df['comp'][idx] = '1'

df['fltg'] = 0
idx = df[df['fltg_12m'] > (round(df['fltg_12m'].median(), 4))].index
df['fltg'][idx] = '1'

df['lpop'] = 0
idx = df[df['lpop_12m'] > (round(df['lpop_12m'].median(), 4))].index
df['lpop'][idx] = '1'

C:\Users\duPar\AppData\Local\Temp\ipykernel_7808\85463361.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sale'][idx] = '1'
C:\Users\duPar\AppData\Local\Temp\ipykernel_7808\85463361.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comp'][idx] = '1'
C:\Users\duPar\AppData\Local\Temp\ipykernel_7808\85463361.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fltg'][idx] = '1'
C:\Users\duPar\AppData\Local\Temp\ipykerne

In [43]:
df

,pnu,sale_12m,comp_12m,fltg_12m,lpop_12m,base_ym,sale,comp,fltg,lpop
0,1111010100100010000,0.9957,0.8889,1.2237,1.5722,202110,0,0,1,1
1,1111010100100030065,0.9569,1.5000,1.2237,1.5722,202110,0,1,1,1
2,1111010100100060003,1.1285,0.5000,3.2260,2.4503,202110,1,0,1,1
3,1111010100100150004,0.9467,1.0000,1.2237,1.5722,202110,0,0,1,1
4,1111010100100150005,0.9577,2.0000,1.2237,1.5722,202110,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...
10736287,4137011300106020001,1.1168,0.5000,1.6687,1.2762,202209,1,0,1,1
10736288,4137011300106030002,0.9701,1.0000,1.6687,1.2762,202209,0,0,1,1
10736289,4137011300106050003,0.9656,1.0000,1.6687,1.2762,202209,0,0,1,1
10736290,4137011300106050006,0.9429,1.0000,1.6687,1.2762,202209,0,0,1,1


In [42]:
df['lpop'].value_counts()

0    7121450
1    3614842
Name: lpop, dtype: int64

In [3]:
df_bld_principal_use

,principal_use,pnu
0,제1종근린생활시설,1111010300100740006
1,제2종근린생활시설,1111014200100480000
2,제2종근린생활시설,1111017400103920006
3,제1종근린생활시설,1111018300103000001
4,제2종근린생활시설,1114011300100880002
...,...,...
47452984,단독주택,4886035026100950015
47452985,단독주택,4887036022100410000
47452986,단독주택,5011025922117530000
47452987,단독주택,5013011600139320007


In [ ]:
# 데이터 프레임 병합
df = pd.merge(df_f_tot, df_bld_rgst, on='pnu', how='left')

# 인구가 0명, null인 지역 제외
df = df[~df['bld_count'].isnull()]

# 상업용 부동산 수 / 유동인구
df['commercial_per_population'] = (df['bld_count'] / df['f_tot'])

# 주용도코드 붙이기
df = pd.merge(df, df_bld_principal_use, on='pnu', how='left')
df = df.drop_duplicates('pnu').reset_index(drop=True)

# inf 값 제거후 컬럼 정리
df = df[~(df['commercial_per_population']==np.inf)]
df = df[['pnu', 'principal_use', 'commercial_per_population']]
df['base_ym'] = datetime.today().strftime("%Y%m")

In [ ]:
### 2. DB INSERT
import datetime

table = 'm2.datadam_gentrification'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
     # drop & create
    cursor.execute(
    """
    drop table if exists m2.datadam_gentrification;
    create table m2.datadam_gentrification (    
    pnu varchar(19)
    , principal_use varchar(200)
    , commercial_per_population double precision
    , base_ym varchar(6)
    )
    distkey(pnu)
    compound sortkey(principal_use, base_ym);
    commit;
    """
    )
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))